In [1]:
from astropy.table import Table, column, vstack
import numpy as np
import os
import pandas as pd

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

import sys
sys.path.append('../astro_codes/')

In [2]:
import astropy.units as u
import astropy.constants as const
from scipy.special import kn as K, iv as I

def m_equation(R,R_d,a_b,a_h,M_D,M_B,M_H,X=1.5):
    
    y = (R/(2*R_d)).value
    a = np.exp(2*y)/X
    b = (M_B/M_D) * ((2*y + (3*a_b/R_d)) / (2*y + (a_b/R_d))**3)
    c = (M_H/M_D) * ((2*y + (3*a_h/R_d)) / (2*y + (a_h/R_d))**3)
    d = (y**2/2) * (3*(I(1,y)*K(0,y)) - 3*(I(0,y)*K(1,y)) + (I(1,y)*K(2,y)) - (I(2,y)*K(1,y)))
    e = (4*y) * (I(0,y)*K(0,y) - (I(1,y)*K(1,y)))
    m = a*(b + c + d + e)
    return m

In [395]:
M_b = 4e9 * u.Msun
delta_M_b = 1e9 * u.Msun

M_d = 1e10 * u.Msun
delta_M_d = 1e9 * u.Msun

a_b = 0.6 * u.kpc
delta_a_b = 0.2 * u.kpc

R_d = 3 * u.kpc
delta_R_d = 0.5 * u.kpc

M_hi = 4e9 * u.Msun
delta_M_hi = 1e9 * u.Msun

In [398]:
from uncertainties import ufloat as uf, unumpy as unp
from astropy.nddata import NDDataArray as nda, StdDevUncertainty as sdu
import astropy.units as u

class TotalHalo():
    
    def __init__(self,M_b,delta_M_b,M_d,delta_M_d,a_b,delta_a_b,
                 R_d,delta_R_d,M_hi,delta_M_hi):
        
        self.M_b = M_b.to(u.Msun)
        self.delta_M_b = delta_M_b.to(u.Msun)
        self.M_d = M_d.to(u.Msun)
        self.delta_M_d = delta_M_d.to(u.Msun)
        self.a_b = a_b.to(u.kpc)
        self.delta_a_b = delta_a_b.to(u.kpc)
        self.R_d = R_d.to(u.kpc)
        self.delta_R_d = delta_R_d.to(u.kpc)
        self.M_hi = M_hi.to(u.Msun)
        self.delta_M_hi = delta_M_hi.to(u.Msun)
        
        H0 = 70 * u.km/u.s/u.Mpc
        rho_crit = (3 * H0**2) / (8*math.pi*const.G)
        self.rho_crit = rho_crit.to(u.kg/u.m**3)
        
    def bulge_mass(self):
        return nda(self.M_b,sdu(self.delta_M_b))
    
    def disc_mass(self,hi=True):
        M_disc = nda(self.M_d,sdu(self.delta_M_d))
        if hi is False:
            return M_disc
        else:
            M_hi = nda(self.M_hi,sdu(self.delta_M_hi))
            M_tot = M_disc.add(M_hi)
            return M_tot
    
    def stellar_mass(self,hi=True):
        return self.disc_mass(hi).add(self.bulge_mass())
        
    def halo_mass(self,alpha=-0.5,alpha_upper=-0.475,alpha_lower=-0.575,
                  beta=0,logx0=10.4,gamma=1,
                  logy0=1.61,logy0_lower=1.49,logy0_upper=1.75):

        stellar_mass = uf(self.stellar_mass(hi=False).data,
                          self.stellar_mass(hi=False).uncertainty.array)
        
        alpha = uf(alpha,np.max([alpha_upper-alpha,alpha-alpha_lower]))
        beta = uf(beta,0)
        x0 = uf(10**logx0,0)
        gamma = uf(gamma,0)
        y0 = uf(10**logy0,np.max([10**logy0_upper-10**logy0,
                                  10**logy0-10**logy0_lower]))
    
        a = y0 * (stellar_mass/x0)**alpha
        b = 1/2 + 1/2*((stellar_mass/x0)**gamma)
        c = (beta-alpha)/gamma
        y = a * b**c
        halo_mass = y * stellar_mass
        return nda(halo_mass.nominal_value,
                   sdu(halo_mass.std_dev),unit=u.Msun)
    
    # --- Now for the sizes ---
    def bulge_scale_length(self,scale=1.5,scale_error=0.2):
        scale_factor = nda(scale,sdu(scale_error))
        r_b = nda(a_b,sdu(delta_a_b))
        r_b_mod = r_b.divide(scale_factor)
        return r_b.divide(scale_factor)
        
    def halo_scale_length(self):
        K = 3 / (4*np.pi*200*self.rho_crit)
        M200 = self.halo_mass()
        M200_value = M200.data * (u.Msun)
        M200_error = M200.uncertainty.array * (u.Msun)
        logc200 = 0.905 - 0.101 * np.log10(M200_value/(10**12 * u.Msun))
        c200 = 10**logc200
        r200 = (K*(M200_value))**(1/3)
        r200 = r200.to(u.kpc)
        r200_error = (1/3) * r200 * M200_error/M200_value
        r_s = nda(r200,sdu(r200_error))
        return r_s.divide(c200)
    
    def hi_scale_length(self,m=1.87,m_error=0.03,c=7.2,c_error=0.03,
                        k=0.19,k_error=0.03): # Wang+14, Lelli+16
        m_param = uf(m,m_error)
        c_param = uf(c,c_error)
        k_param = uf(k,k_error)
        hi_mass = uf(self.M_hi.value,self.delta_M_hi.value)
        loghimass = unp.log10(hi_mass)
        logr_hi = (loghimass - c_param) / m
        r_hi = 10**logr_hi
        r_s = k_param * r_hi
        return nda(r_s.nominal_value,sdu(r_s.std_dev),unit=u.kpc)
    
    def stellar_disc_scale_length(self,scale=1.5,scale_error=0.2):
        scale_factor = nda(scale,sdu(scale_error))
        r_d = nda(R_d,sdu(delta_R_d))
        r_d_mod = r_d.divide(scale_factor)
        return r_d.divide(scale_factor)
    
    def disc_scale_length(self): 
        r_d = uf(self.stellar_disc_scale_length().data,
                 self.stellar_disc_scale_length().uncertainty.array)
        r_g = uf(self.hi_scale_length().data,
                 self.hi_scale_length().uncertainty.array)
        
        rho_d = uf(self.M_hi.value,self.delta_M_hi.value)/r_d
        rho_g = uf(self.disc_mass(hi=False).data,
                   self.disc_mass(hi=False).uncertainty.array)/r_g
        
        r_s = 1 / (unp.log((rho_d/(rho_g+rho_d)*unp.exp(1/r_g)) 
                         + (rho_g/(rho_g+rho_d)*unp.exp(1/r_d))))
        
        return nda(r_s.nominal_value,sdu(r_s.std_dev),unit=u.kpc)
        

In [407]:
halo = TotalHalo(M_b,delta_M_b,M_d,delta_M_d,a_b,delta_a_b,
                 R_d,delta_R_d,M_hi,delta_M_hi)

def printer(data,name):
    print('{}: {} +- {} ({}%)'.format(name,data.data,data.uncertainty.array,
                              np.round(data.uncertainty.array/data.data*100,2)))
    return None

names = ('bulge mass','disc mass','halo mass',
         'bulge scale length','disc scale length','halo scale length')

datas = (halo.bulge_mass(),halo.disc_mass(),halo.halo_mass(),
         halo.bulge_scale_length(),halo.disc_scale_length(),
         halo.halo_scale_length())

for d, n in zip(datas,names):
    printer(d,n)

bulge mass: 4000000000.0 +- 1000000000.0 (25.0%)
disc mass: 14000000000.0 +- 1414213562.373095 (10.1%)
halo mass: 674127647673.7296 +- 261110175881.5145 (38.73%)
bulge scale length: 0.39999999999999997 +- 0.14360439485692011 (35.9%)
disc scale length: 2.436536757629943 +- 0.393374240404731 (16.14%)
halo scale length: 21.632776495805 +- 2.793011133285629 (12.91%)


#### Check for a mass systematic:

In [26]:
#gz2_data = Table.read('../fits/full_sample_debiased_w_low_z_mod.fits')
#axial_data = Table.read('../fits/Axial_ratios/dr7_isoAB_matched.fits')
#mendel_data = Table.read('../sparcfire2/fits/mendel_matched.fits')

In [385]:
unp.mean()

AttributeError: 'module' object has no attribute 'mean'